In [1]:
import json

In [2]:
with open("C:/Projects/ThermalNetwork/demos/example_project_combine_GHE_2.json", "r") as f:
    json_data2 = json.load(f)

In [3]:
with open("C:/Projects/ThermalNetwork/demos/example_project_combine_GHE.json", "r") as f:
    json_data = json.load(f)

In [4]:
geojson_data = json_data
geojson_data2 = json_data2

In [5]:

#THIS IS GOOD
def find_startloop_feature_id(features):
    for feature in features:
        if feature['properties'].get('startLoop') == 'true':
            start_feature_id = feature['properties'].get('buildingId') or feature['properties'].get('DSId')
            return start_feature_id
    return None

def get_connected_features(geojson_data):
    features = geojson_data['features']
    connectors = [feature for feature in features if feature['properties']['type'] == 'ThermalConnector']
    connected_features = []

    #get the id of the building or ds from the thermaljunction that has startloop: true
    startloop_feature_id = find_startloop_feature_id(features)
    
    # Start with the first connector
    start_feature_id = connectors[0]['properties']['startFeatureId']
    connected_features.append(start_feature_id)

    while True:
        next_feature_id = None
        for connector in connectors:
            if connector['properties']['startFeatureId'] == connected_features[-1]:
                next_feature_id = connector['properties']['endFeatureId']
                break

        if next_feature_id:
            connected_features.append(next_feature_id)
            if next_feature_id == start_feature_id:
                break
        else:
            break

    # Filter and return the building and district system features
    connected_objects = []
    for feature in features:
        feature_id = feature['properties']['id']
        if feature_id in connected_features and feature['properties']['type'] in ['Building', 'District System']:
            connected_objects.append({
                'id': feature_id,
                'type': feature['properties']['type'],
                'name': feature['properties'].get('name', ''),
                'start_loop': 'true' if feature_id == startloop_feature_id else None
            })

    return connected_objects

def reorder_connected_features(features):
    while features[0].get('start_loop') != 'true':
        features.append(features.pop(0))
    return features

In [6]:
connected_features = get_connected_features(geojson_data2)
for feature in connected_features:
    print(feature)
reordered_features = reorder_connected_features(connected_features)
reordered_features

{'id': '8', 'type': 'Building', 'name': 'Hospital 1', 'startloop': None}
{'id': '9', 'type': 'Building', 'name': 'Hospital 2', 'startloop': None}
{'id': '10', 'type': 'Building', 'name': 'Mixed use 2', 'startloop': 'true'}
{'id': '0b575a8f-97d1-47e6-b329-7ef7566d26f2', 'type': 'District System', 'name': 'New District System_1', 'startloop': None}
{'id': '15a25e2f-c425-46e9-8036-b1d0d09127e2', 'type': 'District System', 'name': 'New District System_2', 'startloop': None}


[{'id': '10', 'type': 'Building', 'name': 'Mixed use 2', 'startloop': 'true'},
 {'id': '0b575a8f-97d1-47e6-b329-7ef7566d26f2',
  'type': 'District System',
  'name': 'New District System_1',
  'startloop': None},
 {'id': '15a25e2f-c425-46e9-8036-b1d0d09127e2',
  'type': 'District System',
  'name': 'New District System_2',
  'startloop': None},
 {'id': '8', 'type': 'Building', 'name': 'Hospital 1', 'startloop': None},
 {'id': '9', 'type': 'Building', 'name': 'Hospital 2', 'startloop': None}]